In [ ]:
import os
import xml.dom.minidom
import xml.etree.ElementTree as ET
from PIL import Image

In [ ]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xcenter, ycenter = float(tokens[2]), float(tokens[3])
            w, h = float(tokens[4]), float(tokens[5])
            xmin, ymin = int(xcenter - w/2), int(ycenter - h/2)
            xmax, ymax = int(xcenter + w/2), int(ycenter + h/2)
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml(basename, labels, data_path, save_path):
    jpg_name = os.path.join(data_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    for label in labels:
        object = ET.SubElement(root, "object")
        ET.SubElement(object, "name").text = label["name"]
        ET.SubElement(object, "pose").text = "Unspecified"
        ET.SubElement(object, "truncated").text = "0"
        ET.SubElement(object, "difficult").text = "0"
        bndbox = ET.SubElement(object, "bndbox")          
        ET.SubElement(bndbox, "xmin").text = str(label["xmin"])
        ET.SubElement(bndbox, "ymin").text = str(label["xmin"])
        ET.SubElement(bndbox, "xmax").text = str(label["xmin"])
        ET.SubElement(bndbox, "ymax").text = str(label["xmin"])

    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))
        
        
def main(result_txt, class_i, data_path, save_path):
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml(basename, labels, data_path, save_path)
        print("generated", basename)

In [ ]:
result_txt = ""
class_i = ""
data_path = ""
save_path = ""